In [39]:
import pandas as pd
import json
import statistics
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# 📌 Extracción

Para iniciar mi análisis, importaré los datos de la API de Telecom X. Estos datos están disponibles en formato JSON y contienen información esencial sobre los clientes, incluyendo datos demográficos, tipo de servicio contratado y estado de evasión.

✅ Cargando los datos directamente desde la API utilizando Python.

✅ Normalizando las columnas con diccionarios.

✅ Convirtiendo los datos a un DataFrame de Pandas para facilitar su manipulación.

In [40]:
with open('../data/TelecomX_Data.json','r') as f:
    data = json.loads(f.read())

In [41]:
pd_data = pd.DataFrame(data)

# 🔧 Transformación

## Conociendo el conjunto de datos

Ahora que extraí los datos, es fundamental comprender la estructura del dataset y el significado de sus columnas. Esta etapa ayudará a identificar qué variables son más relevantes para el análisis de evasión de clientes.

📌 Para facilitar este proceso, hay en el README.md un diccionario de datos con la descripción de cada columna.

¿Qué debo hacer?

✅ Explorar las columnas del dataset y verificar sus tipos de datos.

✅ Consultar el diccionario para comprender mejor el significado de las variables.

✅ Identificar las columnas más relevantes para el análisis de evasión.



In [42]:
pd_data.sample(3)

,customerID,Churn,customer,phone,internet,account
1614,2292-XQWSV,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'No', 'MultipleLines': 'No ph...","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
3075,4290-BSXUX,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4716,6464-KEXXH,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'No', 'MultipleLines': 'No ph...","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


## Normalizando nuestros datos

Vemos que las columnas poseen diccionarios, por lo que normalizaré para separarlos en nuevas columnas, almacenandolo en un nuevo DataFrame para poder analizar y comparar los datos.

In [43]:
data_norm = pd.json_normalize(data)

In [44]:
data_norm.sample(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
1921,2718-YSKCS,No,Male,0,Yes,Yes,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.60,1387.45
735,1051-GEJLJ,No,Female,0,No,No,2,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.50,27.55
6606,9081-WWXKP,No,Female,0,Yes,Yes,47,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),20.25,873.4


Vemos los tipos de datos almacenados en las columnas

In [45]:
data_norm.dtypes

customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total         object
dtype: object

Observamos que la columna **account.Charges.Total** tiene datos de tipo *object*, entonces la convertimos a tipo *float64*.

In [46]:
data_norm['account.Charges.Total'] = pd.to_numeric(data_norm['account.Charges.Total'], errors='coerce')

In [47]:
data_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

Observamos que nuestro df es de 7267 filas por 21 columnas.

In [48]:
data_norm.shape

(7267, 21)

En total tenemos 152607 datos

In [49]:
data_norm.size

152607

Vemos una lista de las columnas.

In [50]:
data_norm.columns

Index(['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen',
       'customer.Partner', 'customer.Dependents', 'customer.tenure',
       'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService',
       'internet.OnlineSecurity', 'internet.OnlineBackup',
       'internet.DeviceProtection', 'internet.TechSupport',
       'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract',
       'account.PaperlessBilling', 'account.PaymentMethod',
       'account.Charges.Monthly', 'account.Charges.Total'],
      dtype='object')

## Comprobación de incoherencias en los datos

En este paso, verifico si hay problemas en los datos que puedan afectar el análisis. Prestando atención a valores ausentes, duplicados, errores de formato e inconsistencias en las categorías. Este proceso es esencial para asegurarme de que los datos estén listos para las siguientes etapas.

In [51]:
data_norm.describe()

,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly,account.Charges.Total
count,7267.000000,7267.000000,7267.000000,7256.000000
mean,0.162653,32.346498,64.720098,2280.634213
std,0.369074,24.571773,30.129572,2268.632997
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.425000,400.225000
50%,0.000000,29.000000,70.300000,1391.000000
75%,0.000000,55.000000,89.875000,3785.300000
max,1.000000,72.000000,118.750000,8684.800000


Vemos que 'Churn' tiene 3 valores posibles, cuando sólo deberían ser 2.

In [52]:
data_norm.nunique()

customerID                   7267
Churn                           3
customer.gender                 2
customer.SeniorCitizen          2
customer.Partner                2
customer.Dependents             2
customer.tenure                73
phone.PhoneService              2
phone.MultipleLines             3
internet.InternetService        4
internet.OnlineSecurity         3
internet.OnlineBackup           3
internet.DeviceProtection       3
internet.TechSupport            3
internet.StreamingTV            3
internet.StreamingMovies        3
account.Contract                3
account.PaperlessBilling        2
account.PaymentMethod           4
account.Charges.Monthly      1585
account.Charges.Total        6530
dtype: int64

Vemos los valores únicos por columna para corroborar valores

In [53]:
for columna in data_norm.columns:
    print(f'{columna} \t {data_norm[columna].unique()}')

customerID 	 ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn 	 ['No' 'Yes' '']
customer.gender 	 ['Female' 'Male']
customer.SeniorCitizen 	 [0 1]
customer.Partner 	 ['Yes' 'No']
customer.Dependents 	 ['Yes' 'No']
customer.tenure 	 [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
phone.PhoneService 	 ['Yes' 'No']
phone.MultipleLines 	 ['No' 'Yes' 'No phone service']
internet.InternetService 	 ['DSL' 'Fiber optic' 'No' 'Fibjsoner optic']
internet.OnlineSecurity 	 ['No' 'Yes' 'No internet service']
internet.OnlineBackup 	 ['Yes' 'No' 'No internet service']
internet.DeviceProtection 	 ['No' 'Yes' 'No internet service']
internet.TechSupport 	 ['Yes' 'No' 'No internet service']
internet.StreamingTV 	 ['Yes' 'No' 'No internet service']
internet.StreamingMovies 	 ['No' 'Yes' 'No 

⚠️ En 'Churn' no debería haber valores vacíos.

Verificamos duplicados de filas

In [54]:
sum(int(x) for x in data_norm.duplicated().values)

0

No hay filas duplicadas

Verificamos si hay un sólo registro para cada cliente

In [55]:
len(data_norm['customerID'].unique()) == len(data_norm)

True

No hay más de un registro para un cliente

Verificamos si hay valores NaN en cada columna numérica

In [56]:
for columna in ['customer.SeniorCitizen', 'customer.tenure', 'account.Charges.Monthly', 'account.Charges.Total']:
    print(columna, data_norm[columna].isna().sum())


customer.SeniorCitizen 0
customer.tenure 0
account.Charges.Monthly 0
account.Charges.Total 11


⚠️ Hay valores NaN en la columna *account.Charges.Total*

## Manejo de Inconsistencias

Aplico las correcciones necesarias. Ajusto los datos para asegurarme de que estén completos y coherentes, preparándolos para las siguientes etapas del análisis.

In [57]:
data_norm.sample(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
2774,3842-QTGDL,No,Male,0,Yes,No,16,Yes,No,Fiber optic,...,Yes,No,No,No,No,Month-to-month,No,Bank transfer (automatic),80.75,1321.30
5277,7206-PQBBZ,No,Male,1,Yes,No,11,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,834.70
1441,2073-QBVBI,No,Female,0,Yes,No,41,No,No phone service,DSL,...,Yes,No,Yes,No,No,One year,No,Mailed check,35.45,1391.65


Buscamos la fila con 'Churn' == '' y las de 'account.Charges.Total' == NaN

In [58]:
data_norm[data_norm['Churn']=='']

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.30
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.10
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.20
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,0,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.20
7180,9872-RZQQB,,Female,0,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,0,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,0,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.90


In [59]:
data_norm[data_norm['account.Charges.Total'].isna()]

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


Eliminaremos esos registros. Creamos una lista con sus índices

In [60]:
filas_a_eliminar = set(data_norm[data_norm['Churn']==''].index) | set(data_norm[data_norm['account.Charges.Total'].isna()].index)

In [61]:
filas_a_eliminar

{30,
 75,
 96,
 98,
 175,
 219,
 312,
 351,
 368,
 374,
 380,
 382,
 395,
 439,
 451,
 495,
 540,
 590,
 640,
 669,
 681,
 739,
 791,
 842,
 876,
 877,
 903,
 912,
 932,
 973,
 975,
 992,
 1013,
 1017,
 1160,
 1172,
 1218,
 1236,
 1303,
 1364,
 1366,
 1517,
 1657,
 1705,
 1764,
 1775,
 1795,
 1805,
 1825,
 1860,
 1883,
 1955,
 2021,
 2075,
 2101,
 2138,
 2151,
 2154,
 2158,
 2200,
 2232,
 2245,
 2264,
 2308,
 2390,
 2394,
 2429,
 2467,
 2494,
 2576,
 2584,
 2613,
 2627,
 2644,
 2690,
 2726,
 2733,
 2751,
 2879,
 2913,
 2919,
 2930,
 2945,
 2953,
 2973,
 2989,
 3053,
 3060,
 3076,
 3134,
 3177,
 3199,
 3202,
 3203,
 3207,
 3220,
 3249,
 3266,
 3290,
 3300,
 3305,
 3320,
 3365,
 3378,
 3438,
 3468,
 3538,
 3590,
 3617,
 3619,
 3688,
 3724,
 3804,
 3827,
 3833,
 3844,
 3858,
 3900,
 3924,
 3968,
 4021,
 4072,
 4081,
 4128,
 4169,
 4196,
 4199,
 4282,
 4327,
 4390,
 4393,
 4396,
 4411,
 4413,
 4431,
 4497,
 4541,
 4578,
 4579,
 4599,
 4609,
 4662,
 4665,
 4713,
 4750,
 4753,
 4762,
 4769,


Eliminamos las filas según los índices almacenados, y reseteamos el index

In [62]:
data_norm.drop(filas_a_eliminar, inplace=True)
data_norm.reset_index(drop=True, inplace=True)

Verifico si no quedaron registros con 'Churn' == '', y si la cantidad restante es correcta

In [63]:
( len(data_norm[data_norm['Churn']==''].index) == 0 ) & ( len(data_norm) == ( len(pd_data) - len(filas_a_eliminar) ) )

True

## Columna de cuentas diarias

Ahora que los datos están limpios, creamos la columna "Cuentas_Diarias" solicitada por el desafío, bajo el nombre "account.Charges.Daily". Usamos la facturación mensual para calcular el valor diario, proporcionando una visión más detallada del comportamiento de los clientes a lo largo del tiempo.

In [64]:
data_norm['account.Charges.Daily'] = (data_norm['account.Charges.Monthly'] / 30).round(2)
data_norm.head(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


## Estandarización y Transformación de Datos

La estandarización y transformación de datos es una etapa opcional, pero altamente recomendada, ya que busca hacer que la información sea más consistente, comprensible y adecuada para el análisis. Durante esta fase, por ejemplo, podemos convertir valores textuales como "Sí" y "No" en valores binarios (1 y 0), lo que facilita el procesamiento matemático y la aplicación de modelos analíticos.

Además, traducir o renombrar columnas y datos hace que la información sea más accesible y fácil de entender, especialmente cuando se trabaja con fuentes externas o términos técnicos. Aunque no es un paso obligatorio, puede mejorar significativamente la claridad y comunicación de los resultados, facilitando la interpretación y evitando confusiones, especialmente al compartir información con stakeholders no técnicos.

Los nombres de las columnas decido dejarlos en inglés, me resulta más compacto a la vista, que su correspondiente al español.

Pero sí modificaré las columnas:
```
{
    'Churn': {'No': 'Active', 'Yes': 'Churned'},
    'customer.SeniorCitizen': {0: 'No', 1: 'Yes'}
}
```


In [65]:
diccionario_reemplazo_multiple = {
    'Churn': {'No': 'Active', 'Yes': 'Churned'},
    'customer.SeniorCitizen': {0: 'No', 1: 'Yes'}
}


In [66]:
data_norm = data_norm.replace(diccionario_reemplazo_multiple);
data_norm.sample(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
6287,8909-BOLNL,Active,Male,No,No,No,3,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,21.20,52.05,0.71
1403,2075-PUEPR,Active,Male,No,Yes,Yes,55,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.35,1153.25,0.64
3626,5146-YYFRZ,Active,Male,No,No,No,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.95,1311.75,0.66


#📊 Carga y análisis

## Análisis Descriptivo

Para comenzar, realizamos un análisis descriptivo de los datos, calculando métricas como media, mediana, desviación estándar y otras medidas que ayuden a comprender mejor la distribución y el comportamiento de los clientes.

In [67]:
data_norm.describe()

,customer.tenure,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,32.421786,64.798208,2283.300441,2.159891
std,24.545260,30.085974,2266.771362,1.002955
min,1.000000,18.250000,18.800000,0.610000
25%,9.000000,35.587500,401.450000,1.187500
50%,29.000000,70.350000,1397.475000,2.340000
75%,55.000000,89.862500,3794.737500,2.992500
max,72.000000,118.750000,8684.800000,3.960000


Iniciaremos evaluando:
* *customer.tenure*: Tiempo que el cliente ha estado en la empresa. Es una variable continua y muy relevante para entender lealtad o permanencia.

* *account.Charges.Monthly*: Monto mensual que paga el cliente. Es clave para ver si los clientes que pagan más tienden a irse o quedarse.

* *account.Charges.Total*: Total acumulado pagado por el cliente. Da una idea del valor del cliente para la empresa.

* *account.Charges.Daily*: Probablemente una derivada del total dividido por los días de permanencia. También puede ser útil para comparar con otras métricas de consumo.

In [68]:
def analisis_Descriptivo(columna):
    print(f'Los valores mínimo y máximo respectivamente de {columna} es de {data_norm[columna].min()} y {data_norm[columna].max()}')
    print(f'El promedio de {columna} es de {data_norm[columna].mean()}')
    print(f'La mediana de {columna} es de {data_norm[columna].median()}')
    print(f'La desviación estándar de {columna} es de {statistics.pstdev(data_norm[columna])}\n')


In [69]:
analisis_Descriptivo('customer.tenure')
analisis_Descriptivo('account.Charges.Monthly')
analisis_Descriptivo('account.Charges.Total')
analisis_Descriptivo('account.Charges.Daily')

Los valores mínimo y máximo respectivamente de customer.tenure es de 1 y 72
El promedio de customer.tenure es de 32.421786120591584
La mediana de customer.tenure es de 29.0
La desviación estándar de customer.tenure es de 24.543514392682013

Los valores mínimo y máximo respectivamente de account.Charges.Monthly es de 18.25 y 118.75
El promedio de account.Charges.Monthly es de 64.79820819112628
La mediana de account.Charges.Monthly es de 70.35
La desviación estándar de account.Charges.Monthly es de 30.083834589143024

Los valores mínimo y máximo respectivamente de account.Charges.Total es de 18.8 y 8684.8
El promedio de account.Charges.Total es de 2283.3004408418656
La mediana de account.Charges.Total es de 1397.475
La desviación estándar de account.Charges.Total es de 2266.6101807145346

Los valores mínimo y máximo respectivamente de account.Charges.Daily es de 0.61 y 3.96
El promedio de account.Charges.Daily es de 2.1598905005688285
La mediana de account.Charges.Daily es de 2.34
La des

## Distribución de evasión

En este paso, el objetivo es comprender cómo está distribuida la variable "churn" (evasión) entre los clientes. Utilizaré un gráfico de barras para visualizar la proporción de clientes que permanecieron y los que se dieron de baja.

In [34]:
import matplotlib.pyplot as plt

In [35]:
data_norm.head(2)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
0,0002-ORFBO,Active,Female,No,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3,2.19
1,0003-MKNFE,Active,Male,No,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4,2.00


In [122]:
def comp_churn(df, column, title, xtitle, barNorm = None, out = True):
    fig = px.histogram(df, 
                    x = column, 
                    text_auto = True, 
                    color = 'Churn', 
                    barmode='relative', 
                    barnorm = barNorm,
                    title = title,
                    color_discrete_sequence=['green', 'red']
                    )
    fig.update_layout(
        title_x=0.5, # Centrar el título
        xaxis_title= xtitle,
        yaxis_title= 'Churn rate',
        bargap=0.2
    )
    fig.show()
    
    if out == True:
        print(title)
        for index, value in df[column][df['Churn'] == 'Churned'].value_counts().items():
            print(f"{index}: {(value * 100 / df[column][df[column] == index].value_counts().sum()):0.3f} %")
    

## Recuento de evasión por variables categóricas

Ahora, exploraremos cómo se distribuye la evasión según variables categóricas, como género, tipo de contrato, método de pago, entre otras.

Este análisis puede revelar patrones interesantes, por ejemplo, si los clientes de ciertos perfiles tienen una mayor tendencia a cancelar el servicio, lo que ayudará a orientar acciones estratégicas.

In [ ]:
data_norm.head(2)

In [123]:
comp_churn(data_norm, 'customer.gender', 'Churn Rate by Gender', 'Gender')

Churn Rate by Gender
Female: 26.960 %
Male: 26.205 %


In [124]:
comp_churn(data_norm, 'customer.SeniorCitizen', 'Churn Rate by Senior Citizen Status', 'Senior Citizen')

Churn Rate by Senior Citizen Status
No: 23.650 %
Yes: 41.681 %


In [125]:
comp_churn(data_norm, 'customer.tenure', 'Churn rate by customer tenure', 'Customer tenure')

Churn rate by customer tenure
1: 61.990 %
2: 51.681 %
3: 47.000 %
4: 47.159 %
5: 48.120 %
7: 38.931 %
9: 38.655 %
10: 38.793 %
8: 34.146 %
6: 36.364 %
12: 32.479 %
13: 34.862 %
15: 37.374 %
11: 31.313 %
16: 35.000 %
22: 30.000 %
17: 29.885 %
14: 31.579 %
18: 24.742 %
24: 24.468 %
25: 29.114 %
32: 27.536 %
19: 26.027 %
20: 25.352 %
21: 26.984 %
31: 24.615 %
30: 22.222 %
43: 23.077 %
49: 22.727 %
37: 23.077 %
29: 20.833 %
26: 18.987 %
35: 17.045 %
33: 21.875 %
41: 20.000 %
39: 25.000 %
53: 20.000 %
42: 21.538 %
47: 20.588 %
38: 22.034 %
66: 14.607 %
27: 18.056 %
40: 20.312 %
54: 19.118 %
23: 15.294 %
46: 16.216 %
28: 21.053 %
34: 18.462 %
58: 16.418 %
70: 9.244 %
50: 14.706 %
56: 12.500 %
36: 20.000 %
67: 10.204 %
48: 14.062 %
65: 11.842 %
68: 9.000 %
55: 14.062 %
52: 10.000 %
61: 10.526 %
51: 11.765 %
59: 13.333 %
69: 8.421 %
57: 12.308 %
71: 3.529 %
60: 7.895 %
72: 1.657 %
45: 9.836 %
44: 11.765 %
62: 7.143 %
63: 5.556 %
64: 5.000 %


In [126]:
df_grouped = data_norm.groupby(['customer.tenure', 'Churn']).size().unstack(fill_value=0)

df_grouped['churn_rate'] = (df_grouped.get('Active', 0) / df_grouped.sum(axis=1)) * 100

px.line(df_grouped.reset_index(),
        x='customer.tenure',
        y='churn_rate',
        markers=True,
        title='Churn Rate by Customer Tenure')


In [ ]:
px.scatter(data_norm,
           x="customer.tenure",
           y="account.Charges.Monthly",
           color="Churn",
           title="Churn by Tenure vs Monthly Charges")

In [ ]:
conclusiones.append('Se observa que a mayor permanencia en la empresa, los cargos mensuales aumentan o se van dando de baja')

In [ ]:
comp_churn(data_norm, 'account.Contract', "Customer Churn by Contract Type", 'Contract Type')

In [ ]:
conclusiones.append('Se observa una mayor tasa de evasión en los contratos de tipo mes a mes, luego en un valor mucho menor la de contratos anuales, y la de contratos a dos años es mínima')

In [ ]:
px.box(data_norm, x = 'Churn', y = 'account.Charges.Monthly', color = 'Churn', color_discrete_sequence=['green', 'red'])

In [ ]:
conclusiones.append('Se observa que en los casos de evasión, la mediana de los cargos mensuales es mayor a aquellos clientes que continuan activos.')

In [ ]:
comp_churn(data_norm, 'account.PaymentMethod', "Customer Churn by Payment Method", 'Payment Method')

In [ ]:
conclusiones.append('Se observa mayor evasión por aquellos clientes que utilizan "Electronic check"')

In [ ]:
comp_churn(data_norm, 'customer.Partner', 'Customer Churn by Partner Status', 'Partner Status')

In [ ]:
conclusiones.append('Se observa una tasa de evasión alrededor de un 30% en los clientes sin pareja, mientras que en los que tienen pareja es de menos del 20%')

In [ ]:
comp_churn(data_norm, 'customer.Dependents', 'Customer Churn by Dependents', 'Dependents Status')

In [ ]:
conclusiones.append('Se observa una tasa de evasión levemente superior al 30% en los clientes sin dependiente, mientras que entre los que tienen dependientes la tasa es de alrededor ')

In [ ]:
comp_churn(data_norm, 'phone.PhoneService', '', '')
comp_churn(data_norm, 'phone.MultipleLines', '', '')
comp_churn(data_norm, 'internet.InternetService', '', '')
comp_churn(data_norm, 'internet.DeviceProtection', '', '')
comp_churn(data_norm, 'internet.TechSupport', '', '')
comp_churn(data_norm, 'internet.TechSupport', '', '')
comp_churn(data_norm, 'internet.TechSupport', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')
comp_churn(data_norm, '', '', '')



In [ ]:
data_norm.head(2)

#📄Informe final